In [1]:
import pandas as pd
import math
import numpy as np

import heapq
import numba as nb
from scipy.stats.mstats import gmean, hmean
from state import*
# from state_input_player import*
# from new_verrsion_print import*

# data_full = pd.read_csv('data_test.csv')
# data_full = data_full.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
# all_fomula = np.array(pd.read_csv('congthuc.csv')['fomula'])

data_full

c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[1] = 1
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[2] = 'NOT_INVEST'
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[4:] = np.average(data_full.loc[id: , data_full.columns[4:]], axis=0)
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\st

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
0,90,1.000000,NOT_INVEST,HOSE,8.873960e+12,231283.594397,8.469288e+09,5.072332e+10,8.857091e+05,6.373902e+10,...,6.778475e+11,5.893811e+11,1.097540e+11,3.105095e+11,4.307930e+11,3.586968e+10,1.243129e+10,3.037054e+11,4.984089e+12,-1.218414e+12
1,90,0.000000,AAA,HOSE,4.893114e+12,0.000000,3.926632e+09,5.918750e+10,5.156710e+06,6.215189e+10,...,1.018940e+12,9.794080e+11,2.943622e+10,3.430300e+11,2.816613e+11,3.589115e+10,3.873387e+09,1.870343e+10,5.338777e+12,-1.579764e+12
2,90,0.000000,AAT,HOSE,8.485598e+11,1384.000000,1.576288e+08,5.314787e+09,1.027330e+06,9.241194e+07,...,3.328005e+10,1.410469e+10,2.253305e+10,1.191474e+11,2.649353e+06,5.314787e+09,0.000000e+00,0.000000e+00,3.428906e+11,-1.813202e+11
3,90,0.000000,ABS,HOSE,9.451310e+11,0.000000,6.931579e+08,1.141712e+10,7.377700e+05,8.641370e+07,...,2.160073e+10,1.738495e+11,2.044294e+10,2.653042e+10,1.491120e+09,0.000000e+00,0.000000e+00,2.410694e+09,7.281878e+11,-7.332759e+09
4,90,0.000000,ACC,HOSE,1.911000e+12,258.000000,9.234700e+04,5.818817e+09,3.242300e+05,2.771447e+10,...,7.737720e+11,8.828534e+10,8.608350e+09,1.338862e+10,8.726839e+10,5.818817e+09,0.000000e+00,1.173808e+10,8.545696e+11,-1.158529e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,29,0.829231,GIL,HOSE,2.957500e+11,0.000000,1.178573e+07,4.434805e+08,2.435300e+04,1.918037e+08,...,5.592835e+09,2.042547e+10,1.471138e+08,1.327977e+10,9.091083e+08,2.843084e+08,0.000000e+00,2.477158e+08,4.939735e+10,-1.132837e+10
7120,29,0.792308,VSH,HOSE,8.125000e+12,0.000000,0.000000e+00,4.036377e+09,8.040500e+04,1.217429e+10,...,1.117020e+11,7.003186e+09,4.384513e+09,5.236953e+10,3.844896e+08,3.997361e+09,0.000000e+00,3.763020e+09,5.471286e+11,-1.213501e+12
7121,29,0.741068,FPT,HOSE,2.961458e+13,2550.000000,6.714888e+09,1.364417e+10,1.016800e+05,1.010007e+10,...,5.111802e+11,4.444050e+11,1.211761e+11,4.029315e+11,7.813338e+10,9.609989e+09,0.000000e+00,2.484622e+10,1.477624e+12,-2.382139e+11
7122,29,0.724551,GMD,HOSE,7.643971e+12,0.000000,1.093278e+07,9.889580e+09,1.322200e+05,2.829137e+10,...,1.335678e+12,0.000000e+00,0.000000e+00,5.367310e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.744527e+11,-2.710334e+11


In [2]:
result, per_file = normal_main(player_random, 10,[0])
result, per_file

Average agent and system: 0.1677859044784679 0.16271040409186735


(array([5., 5.]), [0])

In [3]:
result, per_file = normal_main(player_random, 1000,[0])
result, per_file

Average agent and system: 0.18415753496937934 0.1794035620092539


(array([530., 470.]), [0])

In [2]:
ALL_IN4_SYS = np.array([LIST_RANK_NOT_INVEST, np.zeros(ALL_QUARTER), np.zeros(ALL_QUARTER)])
LIST_ALL_COMP_PER_QUARTER = []
for j in range(len(index_test)-1, 0, -1):
    LIST_ALL_COMP_PER_QUARTER.append(index_test[j] - index_test[j-1])
env_state, ALL_IN4_SYS = reset(ALL_IN4_SYS, LIST_ALL_COMP_PER_QUARTER)
env_state[CURRENT_QUARTER_INDEX] =24
player_state = state_to_player(env_state)
player_state


array([ 22.,  41.,  13.,  39.,  36.,   6.,  21.,  10.,  12.,   9.,  43.,
         1.,  31.,  40.,  30.,   7.,   4.,  42.,  34.,  25.,  47.,  29.,
        36.,   9.,  31.,  17.,  46.,  48.,  11.,  50.,  34.,  35.,  21.,
         6.,  16.,   3.,  42.,   4.,  30.,  39.,  88.,  17.,  47.,   2.,
        54.,  79.,  38.,  49.,  68.,  41.,   3.,  46.,  24.,  31.,  18.,
        58.,  32.,  83.,  85.,  89.,  59.,  22.,  41.,  52.,  12.,  32.,
         6.,  55.,  64.,  54.,  21.,   4.,  24.,  58.,  45.,  17.,  70.,
        46.,  42.,   9.,  14.,  12.,  27.,  31.,  29.,  32.,   5.,   4.,
         8.,  33.,  23.,  20.,   6.,  21.,  13.,  28.,  16.,  24.,  17.,
        19.,   3.,  34.,  41.,  26.,  15.,   7.,  10.,  24.,  40.,  45.,
        20.,   1.,  33.,  46.,  32.,  27.,  21.,   4.,  23.,  25.,  20.,
        10.,  32.,  28.,  11.,  45.,  34.,  40.,  30.,   3.,  42.,   5.,
         8.,  29.,  17.,  21.,  35.,  44.,  48.,   7.,  11.,  15.,   5.,
        18.,  10.,   3.,  12.,  21.,   4.,  16.,   

In [9]:
list_all_result = np.array([[0,0]])
for i in range(10):
    list_all_result = np.append(list_all_result, np.array([[1,3]]))
list_all_result

array([0, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3])

In [46]:
env_state[CURRENT_QUARTER_INDEX] = 25
player_state = state_to_player(env_state)
player_state


array([  4.,  42.,  33.,  16.,  35.,   2.,   5.,  28.,  13.,  24.,  44.,
        40.,  46.,  45.,  38.,  25.,  39.,  18.,  43.,  15.,  10.,  87.,
        24.,  85.,   2.,  53.,  89.,  11.,  27.,  83.,  74.,  72.,   7.,
        73.,  25.,   9.,  56.,  42.,   8.,  21.,   4.,   9.,  65.,  68.,
        67.,  17.,  66.,  41.,   6.,  21.,  34.,  53.,  18.,  69.,  47.,
        28.,   3.,  31.,  19.,   2.,  20.,  16.,   2.,  28.,  35.,   7.,
        37.,  10.,  26.,   3.,  18.,  30.,   6.,  12.,   1.,  36.,  23.,
         4.,  21.,  13.,  21.,  25.,  35.,  27.,  19.,  46.,  30.,  31.,
        50.,  36.,   6.,  43.,  28.,  37.,  13.,  12.,   5.,   2.,  17.,
        16.,  35.,  29.,  23.,  30.,  31.,  36.,  33.,  18.,  15.,  21.,
        14.,   4.,   1.,   9.,  19.,  44.,  38.,  46.,  10.,  26.,  13.,
         8.,  12.,  17.,  20.,  19.,   9.,   4.,  11.,  15.,  10.,   3.,
        16.,   2.,   6.,  18.,  14.,   7.,   5.,  21.,  18.,  34.,  61.,
        29.,  52.,  66.,  46.,  12.,   2.,  54.,  2

In [3]:
len(player_state)

965

In [41]:
env_state[CURRENT_QUARTER_INDEX]

23.0

In [22]:
def test_ct(test):
    result_fomula = np.zeros(data_arr.shape[1])
    op = -1
    for item in test:
        if type(item) == int:
            if item == 0:
                op = 0
            else:
                op = 1
        else:
            result_temp = np.zeros(data_arr.shape[1])+1
            for id in range(len(item[0])):
                num = data_arr[item[0][id]]
                denom = data_arr[item[1][id]]
                denom_zero = np.where(denom == 0)[0]
                denom[denom_zero] = 1
                num[denom_zero] = 1
                result_temp =  result_temp*(num/denom)
            if op == 1:
                result_fomula = result_fomula + result_temp
            elif op == 0:
                result_fomula = result_fomula - result_temp
            else:
                raise Exception('toang cmnr')
            op = -1
    return result_fomula

test_ct1 = test_ct(test2)

In [ ]:
data_full[data_full['TIME'] == 30].reset_index(drop=True)

In [ ]:
result, per_file = normal_main(player_random1,  1,[0])
result, per_file

In [40]:
import pandas as pd
import math
import numpy as np
import heapq
from scipy.stats.mstats import gmean, hmean
import numba as nb
import random


data_full_load = pd.read_csv('data_test.csv')
data_full_load = data_full_load.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
def get_index_T(data_full):
    list_T = data_full['TIME']
    index_T = [0]
    for i in range(len(list_T)-1):
        if list_T[i] != list_T[i+1]:
            index_T.append(i+1)
    index_T.append(len(list_T))
    return index_T  
index_test = get_index_T(data_full_load)
data_full = data_full_load.copy()
for id in index_test[:-1]:
    s = data_full_load.iloc[id]
    s[1] = 1
    s[2] = 'NOT_INVEST'
    # s[4:] = np.full(len(data_full.columns) - 4, 1)
    s[4:] = np.average(data_full.loc[id: , data_full.columns[4:]], axis=0)
    data_full = data_full.append(s)
data_full = data_full.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
index_test = np.array(get_index_T(data_full))
data_arr = np.array(data_full[data_full.columns[4:]]).T
NUMBER_VARIABLE = len(data_arr)
all_fomula = np.array(pd.read_csv('congthuc.csv')['fomula'])
TOP_COMP_PER_QUARTER = 20
NUMBER_QUARTER_HISTORY = 24
ALL_QUARTER = len(np.unique(data_full['TIME']))

def get_variable(data_full):
    global A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z, PROFIT, COMPANY
    list_variable = [[]]*26
    list_column = list(data_full.columns)
    PROFIT = np.array(data_full["PROFIT"])
    COMPANY = np.array(data_full["SYMBOL"])
    for i in range(4, len(list_column)):
        list_variable[i-4] = np.array(data_full[list_column[i]])
    [A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z] = list_variable

def save_data():
    np.save('index_test.npy', index_test)
    np.save('data.npy', data_arr)
    np.save('list_rank_profit_not_invest.npy', LIST_RANK_NOT_INVEST)
    np.save('list_company.npy', COMPANY)

get_variable(data_full)

def get_rank_not_invest():
    list_rank_ko_dau_tu = []
    for j in range(len(index_test)-1, 0, -1):
        # profit_q = PROFIT[index_test[j-1]:index_test[j]]
        COMP = COMPANY[index_test[j-1]:index_test[j]]
        list_rank_ko_dau_tu.append(np.where(COMP == 'NOT_INVEST')[0][0]+1)
    return np.array(list_rank_ko_dau_tu)

LIST_RANK_NOT_INVEST = get_rank_not_invest()
LIST_RANK_NOT_INVEST_TEMP = np.zeros(ALL_QUARTER)
LIST_RANK_CT1 = np.zeros(ALL_QUARTER)
LIST_RANK_NOT_INVEST_CT1 = np.zeros(ALL_QUARTER)
LIST_RANK_CT2 = np.zeros(ALL_QUARTER)
LIST_RANK_NOT_INVEST_CT2 = np.zeros(ALL_QUARTER)
LIST_PROFIT_CT1 = np.zeros(ALL_QUARTER)
LIST_PROFIT_CT2 = np.zeros(ALL_QUARTER)


def get_in4_rank_fomula(result_fomula):
    list_rank = []
    list_com = []
    for j in range(len(index_test)-1, 0, -1):
        COMP = COMPANY[index_test[j-1]:index_test[j]]
        rank_thuc = np.argsort(-result_fomula[index_test[j-1]:index_test[j]]) + 1
        list_rank.append(rank_thuc[0])
        list_com.append(COMP[rank_thuc[0]-1])
    return list_rank, list_com

#Hàm này tính cả list_profit
# def get_in4_fomula(fomula):
#     result_ =  np.nan_to_num(eval(fomula), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)
#     list_top_comp = []
#     list_profit = []
#     for j in range(len(index_test)-1, 0, -1):
#         rank_thuc = np.argsort(-result_[index_test[j-1]:index_test[j]]) + 1
#         list_profit.append(PROFIT[index_test[j-1]:index_test[j]][rank_thuc[0]-1])
#         list_top_comp.append(rank_thuc[:TOP_COMP_PER_QUARTER])
#     return np.array(list_top_comp).flatten(), np.array(list_profit)

@nb.njit()
def get_in4_fomula(result_fomula, list_rank_not_invest_temp):
    global LIST_RANK_NOT_INVEST, index_test
    # result_ =  np.nan_to_num(eval(fomula), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)
    list_top_comp = np.array([-1])
    list_rank_not_invest_ct = np.array([-1])
    
    for j in range(len(index_test)-1, 0, -1):
        top2 = heapq.nlargest(2,result_fomula[index_test[j-1]:index_test[j]])         #lấy top 2 giá trị lớn nhất
        if top2[0] == top2[1] or np.max(result_fomula[index_test[j-1]:index_test[j]]) == np.min(result_fomula[index_test[j-1]:index_test[j]]):
            # print('toang cong thuc', top2, np.max(result_fomula[index_test[j-1]:index_test[j]]), np.min(result_fomula[index_test[j-1]:index_test[j]]))
            return np.array([-1]), np.array([-1]), 0
        rank_thuc = np.argsort(-result_fomula[index_test[j-1]:index_test[j]]) + 1
        id_not_invest = LIST_RANK_NOT_INVEST[-j] 
        if list_rank_not_invest_ct[0] == -1:
            list_rank_not_invest_ct = np.array([np.where(rank_thuc == id_not_invest)[0][0]+1])
            list_top_comp = rank_thuc[:TOP_COMP_PER_QUARTER]
        else:
            list_rank_not_invest_ct = np.append(list_rank_not_invest_ct, np.where(rank_thuc == id_not_invest)[0][0]+1)
            list_top_comp = np.append(list_top_comp, rank_thuc[:TOP_COMP_PER_QUARTER])
    list_rank_not_invest_temp = list_rank_not_invest_ct
    return list_top_comp, list_rank_not_invest_temp, 1
    # return np.array(list_top_comp).flatten(), 1

IN4_CT1_INDEX = 0
IN4_CT2_INDEX = ALL_QUARTER*TOP_COMP_PER_QUARTER
HISTORY_AGENT_INDEX= IN4_CT2_INDEX + ALL_QUARTER*TOP_COMP_PER_QUARTER
HISTORY_SYS_BOT_INDEX = HISTORY_AGENT_INDEX + ALL_QUARTER
HISTORY_PROFIT_AGENT = HISTORY_SYS_BOT_INDEX + ALL_QUARTER
ID_NOT_INVEST_CT1 = HISTORY_PROFIT_AGENT + ALL_QUARTER
ID_NOT_INVEST_CT2 = ID_NOT_INVEST_CT1 + 1
CURRENT_QUARTER_INDEX = ID_NOT_INVEST_CT2 + 1
ID_ACTION_INDEX = CURRENT_QUARTER_INDEX + 1
CHECK_END_INDEX = ID_ACTION_INDEX + 1

P_IN4_CT1 = 0
P_IN4_CT2 = P_IN4_CT1 + TOP_COMP_PER_QUARTER*NUMBER_QUARTER_HISTORY
P_GMEAN_P1 = P_IN4_CT2 + TOP_COMP_PER_QUARTER*NUMBER_QUARTER_HISTORY
P_GMEAN_P2 = P_GMEAN_P1 + 1
P_ID_NOT_INVEST_CT1 = P_GMEAN_P2 + 1
P_ID_NOT_INVEST_CT2 = P_ID_NOT_INVEST_CT1 + 1

save_data()

def reset():
    global LIST_RANK_CT1, LIST_RANK_CT2, LIST_PROFIT_CT1, LIST_PROFIT_CT2, LIST_RANK_NOT_INVEST_CT1, LIST_RANK_NOT_INVEST_CT2, LIST_RANK_NOT_INVEST_TEMP
    '''
    Hàm này trả ra 2 công thức và list top20 comp qua từng quý của công thức và các thông tin cần thiết khác
    '''
    # list_fomula = []
    count_fomula = 0
    while count_fomula < 2:
        result_fomula = create_fomula(data_arr)
        temp, LIST_RANK_NOT_INVEST_TEMP, check = get_in4_fomula(result_fomula, LIST_RANK_NOT_INVEST_TEMP)
        # print('check getin4', len(temp), check, result_fomula[:10], temp[:10])
        # print('TEMP, ', LIST_RANK_NOT_INVEST_TEMP[:10])
        count_fomula += check
        if count_fomula == 1 and check == 1:
            LIST_RANK_CT1 = temp.copy()
            LIST_RANK_NOT_INVEST_CT1 = LIST_RANK_NOT_INVEST_TEMP.copy()
            # list_fomula.append(fomula)
        elif count_fomula == 2 and check == 1:
            LIST_RANK_CT2 = temp.copy()
            LIST_RANK_NOT_INVEST_CT2 = LIST_RANK_NOT_INVEST_TEMP.copy()
            # list_fomula.append(fomula)
    id_not_invest_ct1 = LIST_RANK_NOT_INVEST_CT1[0]
    id_not_invest_ct2 = LIST_RANK_NOT_INVEST_CT2[0]
    current_quarter = 0
    id_action = 0
    check_end_game = 0
    history_agent = np.zeros(ALL_QUARTER*3)
    # LIST_RANK_CT1, LIST_PROFIT_CT1 = get_in4_fomula(list_fomula[0])
    # LIST_RANK_CT2, LIST_PROFIT_CT2 = get_in4_fomula(list_fomula[1])
    env_state = np.concatenate((LIST_RANK_CT1, LIST_RANK_CT2, history_agent, np.array([id_not_invest_ct1, id_not_invest_ct2, current_quarter, id_action, check_end_game])))
    return env_state

@nb.njit
def state_to_player(env_state):
    '''
    Hàm này trả ra lịch sử kết quả của 2 công thức trong các quý trước đó
    '''
    id_action = env_state[ID_ACTION_INDEX]
    player_state = np.zeros(2*NUMBER_QUARTER_HISTORY*TOP_COMP_PER_QUARTER + 4)
    player_state[P_ID_NOT_INVEST_CT1] = env_state[ID_NOT_INVEST_CT1]
    player_state[P_ID_NOT_INVEST_CT2] = env_state[ID_NOT_INVEST_CT2]
    if env_state[CURRENT_QUARTER_INDEX] != 0:
        history_ct1 = env_state[max(IN4_CT1_INDEX, TOP_COMP_PER_QUARTER*(env_state[CURRENT_QUARTER_INDEX]-24)):int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER)]
        history_ct2 = env_state[max(IN4_CT2_INDEX, TOP_COMP_PER_QUARTER*(env_state[CURRENT_QUARTER_INDEX]-24)+IN4_CT2_INDEX):int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER)+IN4_CT2_INDEX]
        player_state[P_IN4_CT1:P_IN4_CT1+len(history_ct1)] = history_ct1
        player_state[P_IN4_CT2:P_IN4_CT2+len(history_ct2)] = history_ct2
    agent_history = env_state[HISTORY_AGENT_INDEX : HISTORY_AGENT_INDEX+ALL_QUARTER][:int(env_state[CURRENT_QUARTER_INDEX])]
    sys_bot_history = env_state[HISTORY_SYS_BOT_INDEX : HISTORY_SYS_BOT_INDEX+ALL_QUARTER][:int(env_state[CURRENT_QUARTER_INDEX])]
    if env_state[CHECK_END_INDEX] == 1:
        if id_action == 0:
            player_state[P_GMEAN_P1] = np.exp(np.mean(np.log(agent_history)))
            player_state[P_GMEAN_P2] = np.exp(np.mean(np.log(sys_bot_history)))
        else:
            player_state[P_GMEAN_P1] = np.exp(np.mean(np.log(sys_bot_history)))
            player_state[P_GMEAN_P2] = np.exp(np.mean(np.log(agent_history)))
    return player_state

@nb.njit
def step(action, env_state):
    global LIST_RANK_CT1, LIST_RANK_CT2, LIST_PROFIT_CT1, LIST_PROFIT_CT2, LIST_RANK_NOT_INVEST_CT1, LIST_RANK_NOT_INVEST_CT2
    id_action = env_state[ID_ACTION_INDEX]
    result_quarter = 0
    if action == 0:
        result_quarter = LIST_RANK_NOT_INVEST[int(env_state[CURRENT_QUARTER_INDEX])]
        # env_state[int(HISTORY_PROFIT_AGENT+env_state[CURRENT_QUARTER_INDEX])] = 1
    elif action == 1:
        result_quarter = env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER)]
        # env_state[int(HISTORY_PROFIT_AGENT+env_state[CURRENT_QUARTER_INDEX])] = LIST_PROFIT_CT1[int(env_state[CURRENT_QUARTER_INDEX])]
    else:
        result_quarter = env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER+IN4_CT2_INDEX)]
        # env_state[int(HISTORY_PROFIT_AGENT+env_state[CURRENT_QUARTER_INDEX])] = LIST_PROFIT_CT2[int(env_state[CURRENT_QUARTER_INDEX])]
    if result_quarter == 0:
        # print('toang',action,  env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER+IN4_CT2_INDEX)], np.min(LIST_RANK_CT2))
        raise Exception('toang action')
    rank_3_action = np.array([LIST_RANK_NOT_INVEST[int(env_state[CURRENT_QUARTER_INDEX])], env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER)], env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER+IN4_CT2_INDEX)]])
    rank_3_action = np.sort(rank_3_action)
    top_action = np.where(rank_3_action == result_quarter)[0][0] + 1
    # print('quarter', int(env_state[CURRENT_QUARTER_INDEX]),'check', 1/top_action, 'action', action, 'topaction',rank_3_action)
    env_state[int(HISTORY_AGENT_INDEX + ALL_QUARTER*id_action +env_state[CURRENT_QUARTER_INDEX])] = (4-top_action)/3
    if env_state[ID_ACTION_INDEX] == 1:
        env_state[CURRENT_QUARTER_INDEX] += 1  
        #rank giá trị công thức của việc không đầu tư
        if env_state[CURRENT_QUARTER_INDEX] < ALL_QUARTER:
            env_state[ID_NOT_INVEST_CT1] = LIST_RANK_NOT_INVEST_CT1[int(env_state[CURRENT_QUARTER_INDEX])]
            env_state[ID_NOT_INVEST_CT2] = LIST_RANK_NOT_INVEST_CT2[int(env_state[CURRENT_QUARTER_INDEX])]
        env_state[ID_ACTION_INDEX] = 0
    else:
        env_state[ID_ACTION_INDEX] = 1

    return env_state

def action_player(env_state, list_player, temp_file, per_file):
    player_state = state_to_player(env_state)
    current_player = int(env_state[ID_ACTION_INDEX])
    played_move,temp_file,per_file = list_player[current_player](player_state, temp_file, per_file)
    return played_move,temp_file, per_file
    
@nb.njit
def check_winner(env_state):
    agent_history = env_state[HISTORY_AGENT_INDEX : HISTORY_AGENT_INDEX+ALL_QUARTER]
    sys_bot_history = env_state[HISTORY_SYS_BOT_INDEX : HISTORY_SYS_BOT_INDEX+ALL_QUARTER]
    np.exp(np.mean(np.log(sys_bot_history)))
    if np.exp(np.mean(np.log(agent_history))) > np.exp(np.mean(np.log(sys_bot_history))): return 0
    else: return 1

@nb.njit
def check_victory(player_state):
    if not (player_state[P_GMEAN_P1] == player_state[P_GMEAN_P2] and player_state[P_GMEAN_P2] == 0):
        if player_state[P_GMEAN_P1] > player_state[P_GMEAN_P2]: return 1
        else: return 0
    else: return -1

@nb.njit()
def create_fomula(data_arr):
    power = np.random.randint(1, 10)
    operand = np.random.randint(1, 10)
    result_fomula = np.zeros(data_arr.shape[1])
    # ct = []
    for i in range(operand):
        op = np.random.randint(2)
        # ct.append(op)
        numerator = np.random.randint(power, NUMBER_VARIABLE - 1 - power)
        denominator = numerator - power
        numer_var = np.random.randint(1, NUMBER_VARIABLE, numerator)
        result_temp = np.zeros(data_arr.shape[1])+1
        if denominator > 0:
            all_var = np.arange(1,NUMBER_VARIABLE)
            for id in range(len(all_var)):
                if all_var[id] in numer_var:
                    all_var[id] = 0
            all_denom_var = all_var[all_var > 0]
            if len(all_denom_var) < denominator:
                all_denom_var = np.append(all_denom_var, np.random.choice(all_var, denominator - len(all_denom_var)))
            denom_var = np.random.choice(all_denom_var, denominator)
            denom_var = np.append(denom_var, np.zeros(numerator-denominator).astype(np.int64))
            denom_var = denom_var.astype(np.int64)
            # ct.append([numer_var, denom_var])
            for idx in range(len(numer_var)):
                num = data_arr[numer_var[idx]]
                denom = data_arr[denom_var[idx]]
                denom_zero = np.where(denom == 0)[0]
                denom[denom_zero] = 1
                num[denom_zero] = 1
                result_temp =  result_temp*(num/denom)
        else:
            denom_var = np.zeros(numerator).astype(np.int64)
            # ct.append([numer_var, denom_var])
            for id in range(len(numer_var)):
                num = data_arr[numer_var[id]]
                denom = data_arr[denom_var[id]]
                denom_zero = np.where(denom == 0)[0]
                denom[denom_zero] = 1
                num[denom_zero] = 1
                result_temp =  result_temp*(num/denom)
        if op == 1:
            result_fomula = result_fomula + result_temp
        else:
            result_fomula = result_fomula - result_temp
    return result_fomula

def create_fomula_old():
    all_char = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    all_char = 'BCDEFGHIJKLMNOPQRSTUVWXYZ'
    power = np.random.randint(1, 10)
    # print('bậc: ', power)
    operand = np.random.randint(1, 10)
    # print('số toán hạng: ', operand)
    list_exp_child = []
    ct = '('
    for i in range(operand):
        ct += random.choice('+-')
        numerator = random.randint(power, 25-power)
        denominator = numerator - power
        numer_char = random.choices(all_char, k = numerator)
        ct += '(' + '*'.join(numer_char) + ')'
        # print(ct, 'mẫu', denominator)
        if denominator > 0:
            all_denom_char = list(set(numer_char)^set(all_char))
            # print('CHECK: ', len(all_denom_char) , denominator)
            if len(all_denom_char) < denominator:
                all_denom_char += random.choices(all_char, k = (denominator - len(all_denom_char)))
            denom_char = random.choices(all_denom_char, k = denominator)
            ct += '/(' + '*'.join(numer_char) + ')'
    ct += ')' + '/A'*power
    return ct

def one_game(list_player, temp_file, per_file):
    env_state = reset()
    count_turn = 0
    while count_turn < ALL_QUARTER*2:
        action, temp_file, per_file = action_player(env_state, list_player, temp_file, per_file)
        env_state = step(action, env_state)
        count_turn += 1
    env_state[CHECK_END_INDEX] = 1
    for id_player in range(len(list_player)):
        action, temp_file, per_file = action_player(env_state,list_player,temp_file, per_file)
        env_state[ID_ACTION_INDEX] = (env_state[ID_ACTION_INDEX] + 1)%len(list_player)
    result = check_winner(env_state)
    return result, per_file

def normal_main(agent_player, times, per_file):
    global data_full, data_arr
    count = np.zeros(2)
    # all_id_fomula = np.arange(len(all_fomula))
    list_player = [agent_player, player_random]
    for van in range(times):
        temp_file = [[0],[0]]
        # shuffle = np.random.choice(all_id_fomula, 2, replace=False)
        # list_fomula = all_fomula[shuffle]
        winner, file_per = one_game(list_player, temp_file, per_file)
        if winner == 0:
            count[0] += 1
        else:
            count[1] += 1
    return count, file_per

def player_random1(player_state, temp_file, per_file):
    list_action = np.array([0,1,2])
    action = int(np.random.choice(list_action))
    # print('check: ', player_state[P_ID_NOT_INVEST_CT1], player_state[P_ID_NOT_INVEST_CT2])
    check = check_victory(player_state)
    return action, temp_file, per_file

def player_random(player_state, temp_file, per_file):
    list_action = np.array([0,1,2])
    action = int(np.random.choice(list_action))
    # check = check_victory(player_state)
    return action, temp_file, per_file




C:\Users\Admin\AppData\Local\Temp\ipykernel_17996\3948151812.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[1] = 1
C:\Users\Admin\AppData\Local\Temp\ipykernel_17996\3948151812.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[2] = 'NOT_INVEST'
C:\Users\Admin\AppData\Local\Temp\ipykernel_17996\3948151812.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[4:] = np.average(data_full.loc[id: , data_full.columns[4:]], axis=

In [7]:
import numba as nb
import numpy as np

TEST = np.array([1,2,3])
TEST2 = np.arange(3)
print(TEST, TEST2)

@nb.njit()
def test(test):
    # global TEST, TEST2
    a = np.arange(5)
    test = np.append(test, 2)
    return a, test
x, y = test(TEST)
print(x)
print(TEST)

[1 2 3] [0 1 2]
[0 1 2 3 4]
[1 2 3]


In [4]:
TEST2

array([0, 1, 2])

In [6]:
a = np.array(['abc', 'CDF', 'EFG'])
np.save('test.npy', a )

In [7]:
with open('test.npy', 'rb') as f:
    b = np.load(f)
b 

array(['abc', 'CDF', 'EFG'], dtype='<U3')